In [1]:
import sys
import os
import duckdb
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
from config import load_config

c:\Users\longp\FluxForge\config\config.py:18: UserWarning: Field name "schema" in "DatabentoConfig" shadows an attribute in parent "BaseModel"
  class DatabentoConfig(BaseModel):


In [2]:
import logging
from pathlib import Path
from etl.repartitioner import Repartitioner, ParquetCompactor

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [1]:
logger.info("=" * 80)
logger.info("Repartitioning")
logger.info("=" * 80)

repartitioner = Repartitioner(
    source_dir="F:/processed/coinbase/ticker",
    target_dir="F:/processed/coinbase/ticker_new",
    new_partition_cols=['product_id', 'year', 'month', 'day', 'hour'],
)

# Estimate before executing
estimates = repartitioner.estimate_size()
logger.info(f"Estimated partitions: {estimates['num_partitions']}")
logger.info(f"Estimated size: {estimates['estimated_size_mb']:.1f} MB")


NameError: name 'logger' is not defined

In [ ]:
# Execute repartition
stats = repartitioner.repartition(
    delete_source=False,  # Keep source for safety (delete manually later)
    validate=True,        # Verify row counts match
    dry_run=False,        # Set to True to preview only
)

logger.info(f"\n✓ Repartitioning complete!")
logger.info(f"  New location: F:/processed/coinbase/ticker_new")
logger.info(f"  Records: {stats['records_written']:,}")
logger.info(f"  Files: {stats['files_written']}")

In [ ]:
pl_df = pl.scan_parquet("F:/processed/coinbase/ticker/**/*.parquet")
df = pl_df.filter(pl.col("product_id") == "BTC-USD").collect()
df

channel,event_type,product_id,price,volume_24h,low_24h,high_24h,low_52w,high_52w,price_percent_chg_24h,best_bid,best_bid_quantity,best_ask,best_ask_quantity,sequence_num,client_id,server_timestamp,capture_timestamp,year,month,day,hour,minute,second,microsecond,date,datetime,mid_price,spread,spread_bps,range_24h,range_24h_pct,__index_level_0__
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,f64,f64,f64,f64,f64,i64
"""ticker""","""update""","""BTC-USD""",91443.41,11229.986829,89476.35,91925.4,74420.69,126296.0,1.68075,91439.28,0.000033,91439.29,0.668938,7694600,"""""","""2025-11-27T18:01:49.279359723Z""","""2025-11-27T18:01:50.238602+00:…",2025,11,27,18,1,49,279359,"""2025-11-27""","""2025-11-27T18:01:49.279359+00:…",91439.285,0.01,0.001094,2449.05,2.737092,151
"""ticker""","""update""","""BTC-USD""",91441.94,11229.986829,89476.35,91925.4,74420.69,126296.0,1.679115,91439.28,0.000033,91439.29,0.668938,7694678,"""""","""2025-11-27T18:01:49.40466013Z""","""2025-11-27T18:01:50.376705+00:…",2025,11,27,18,1,49,404660,"""2025-11-27""","""2025-11-27T18:01:49.404660+00:…",91439.285,0.01,0.001094,2449.05,2.737092,189
"""ticker""","""update""","""BTC-USD""",91441.94,11229.986829,89476.35,91925.4,74420.69,126296.0,1.679115,91439.28,0.000033,91439.29,0.746184,7694725,"""""","""2025-11-27T18:01:49.526550995Z""","""2025-11-27T18:01:50.485413+00:…",2025,11,27,18,1,49,526550,"""2025-11-27""","""2025-11-27T18:01:49.526550+00:…",91439.285,0.01,0.001094,2449.05,2.737092,208
"""ticker""","""update""","""BTC-USD""",91441.94,11229.986829,89476.35,91925.4,74420.69,126296.0,1.679115,91439.28,0.000033,91439.29,0.705365,7694906,"""""","""2025-11-27T18:01:50.0257487Z""","""2025-11-27T18:01:50.991155+00:…",2025,11,27,18,1,50,25748,"""2025-11-27""","""2025-11-27T18:01:50.025748+00:…",91439.285,0.01,0.001094,2449.05,2.737092,259
"""ticker""","""update""","""BTC-USD""",91439.29,11229.986829,89476.35,91925.4,74420.69,126296.0,1.676169,91439.28,0.000033,91439.29,0.705365,7694956,"""""","""2025-11-27T18:01:50.153558555Z""","""2025-11-27T18:01:51.115351+00:…",2025,11,27,18,1,50,153558,"""2025-11-27""","""2025-11-27T18:01:50.153558+00:…",91439.285,0.01,0.001094,2449.05,2.737092,274
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ticker""","""update""","""BTC-USD""",85678.05,15214.136421,83800.0,91625.54,74420.69,126296.0,-6.439506,85678.05,0.04167,85678.06,0.074767,5009873,"""""","""2025-12-01T20:07:02.660134264Z""","""2025-12-01T20:07:03.224578+00:…",2025,12,1,20,7,2,660134,"""2025-12-01""","""2025-12-01T20:07:02.660134+00:…",85678.055,0.01,0.001167,7825.54,9.338353,29135
"""ticker""","""update""","""BTC-USD""",85680.02,15214.136421,83800.0,91625.54,74420.69,126296.0,-6.437355,85678.05,0.04167,85678.06,0.074767,5010067,"""""","""2025-12-01T20:07:02.897649885Z""","""2025-12-01T20:07:03.462721+00:…",2025,12,1,20,7,2,897649,"""2025-12-01""","""2025-12-01T20:07:02.897649+00:…",85678.055,0.01,0.001167,7825.54,9.338353,29199
"""ticker""","""update""","""BTC-USD""",85680.02,15214.136421,83800.0,91625.54,74420.69,126296.0,-6.437355,85690.0,0.0193,85691.36,0.049575,5010085,"""""","""2025-12-01T20:07:02.910267419Z""","""2025-12-01T20:07:03.475823+00:…",2025,12,1,20,7,2,910267,"""2025-12-01""","""2025-12-01T20:07:02.910267+00:…",85690.68,1.36,0.15871,7825.54,9.338353,29208


# Verify Processed Schemas

Let's inspect the actual column schemas from your processed data:

In [ ]:
ticker_pl = pl.scan_parquet("F:/processed/coinbase/ticker/**/*.parquet")
level2_pl = pl.scan_parquet("F:/processed/coinbase/level2/**/*.parquet")
market_trades_pl = pl.scan_parquet("F:/processed/coinbase/market_trades/**/*.parquet")

In [4]:
# Ticker schema
print("=" * 80)
print("TICKER SCHEMA")
print("=" * 80)
ticker_sample = ticker_pl.limit(1).collect()
print(f"Columns ({len(ticker_sample.columns)}):")
for col in sorted(ticker_sample.columns):
    dtype = ticker_sample[col].dtype
    print(f"  {col:30s} {dtype}")
print(f"\nSample record:")
print(ticker_sample.head(1))

TICKER SCHEMA
Columns (33):
  __index_level_0__              Int64
  best_ask                       Float64
  best_ask_quantity              Float64
  best_bid                       Float64
  best_bid_quantity              Float64
  capture_timestamp              String
  channel                        String
  client_id                      String
  date                           String
  datetime                       String
  day                            Int64
  event_type                     String
  high_24h                       Float64
  high_52w                       Float64
  hour                           Int64
  low_24h                        Float64
  low_52w                        Float64
  microsecond                    Int64
  mid_price                      Float64
  minute                         Int64
  month                          Int64
  price                          Float64
  price_percent_chg_24h          Float64
  product_id                     String
  range

In [5]:
# Level2 schema
print("\n" + "=" * 80)
print("LEVEL2 SCHEMA")
print("=" * 80)
level2_sample = level2_pl.limit(1).collect()
print(f"Columns ({len(level2_sample.columns)}):")
for col in sorted(level2_sample.columns):
    dtype = level2_sample[col].dtype
    print(f"  {col:30s} {dtype}")
print(f"\nSample record:")
print(level2_sample.head(1))


LEVEL2 SCHEMA
Columns (23):
  __index_level_0__              Int64
  capture_timestamp              String
  channel                        String
  client_id                      String
  date                           String
  datetime                       String
  day                            Int64
  event_time                     String
  event_type                     String
  hour                           Int64
  is_snapshot                    Boolean
  microsecond                    Int64
  minute                         Int64
  month                          Int64
  new_quantity                   Float64
  price_level                    Float64
  product_id                     String
  second                         Int64
  sequence_num                   Int64
  server_timestamp               String
  side                           String
  side_normalized                String
  year                           Int64

Sample record:
shape: (1, 23)
┌─────────┬────────────┬──

In [6]:
# Market Trades schema
print("\n" + "=" * 80)
print("MARKET_TRADES SCHEMA")
print("=" * 80)
trades_sample = market_trades_pl.limit(1).collect()
print(f"Columns ({len(trades_sample.columns)}):")
for col in sorted(trades_sample.columns):
    dtype = trades_sample[col].dtype
    print(f"  {col:30s} {dtype}")
print(f"\nSample record:")
print(trades_sample.head(1))


MARKET_TRADES SCHEMA
Columns (24):
  __index_level_0__              Int64
  capture_timestamp              String
  channel                        String
  client_id                      String
  date                           String
  datetime                       String
  day                            Int64
  event_type                     String
  hour                           Int64
  microsecond                    Int64
  minute                         Int64
  month                          Int64
  price                          Float64
  product_id                     String
  second                         Int64
  sequence_num                   Int64
  server_timestamp               String
  side                           String
  side_normalized                String
  size                           Float64
  time                           String
  trade_id                       String
  value                          Float64
  year                           Int64

Sample re

In [7]:
# Summary comparison
print("\n" + "=" * 80)
print("SCHEMA COMPARISON SUMMARY")
print("=" * 80)
print(f"{'Channel':<20} {'Total Columns':<15} {'Key Derived Fields'}")
print("-" * 80)
print(f"{'ticker':<20} {len(ticker_sample.columns):<15} mid_price, spread, spread_bps, range_24h")
print(f"{'level2':<20} {len(level2_sample.columns):<15} is_snapshot, side_normalized")
print(f"{'market_trades':<20} {len(trades_sample.columns):<15} value, side_normalized")
print("=" * 80)

# Check for time partitioning fields
common_fields = ['date', 'year', 'month', 'day', 'hour']
print("\nTime partitioning fields (common to all):")
for field in common_fields:
    ticker_has = field in ticker_sample.columns
    level2_has = field in level2_sample.columns
    trades_has = field in trades_sample.columns
    status = "✓" if all([ticker_has, level2_has, trades_has]) else "✗"
    print(f"  {status} {field}")


SCHEMA COMPARISON SUMMARY
Channel              Total Columns   Key Derived Fields
--------------------------------------------------------------------------------
ticker               33              mid_price, spread, spread_bps, range_24h
level2               23              is_snapshot, side_normalized
market_trades        24              value, side_normalized

Time partitioning fields (common to all):
  ✓ date
  ✓ year
  ✓ month
  ✓ day
  ✓ hour
